In [ ]:
import pickle
import spacy
import random

In [ ]:
data = pickle.load(open('train_data.pkl','rb'))

In [ ]:
data[0]

In [ ]:
nlp = spacy.blank('en')
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner  = nlp.create_pipe('ner')
        nlp.add_pipe(ner , last = True)
        
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Starting Iteration" + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for texts, annotations in train_data:
                try:
                    nlp.update(
                        [texts],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,
                        losses=losses)
                except Exception as e:
                        pass
                    
            print("Losses", losses)

In [ ]:
train_model(data)

In [ ]:
nlp.to_disk('nlp_model')

In [ ]:
nlp_model = spacy.load('nlp_model')

In [ ]:
doc = nlp_model(data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}-{ent.text}')

In [ ]:
! pip install PyMuPDF

In [ ]:
import sys,fitz

In [ ]:
fname = 'Alice Clark CV.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.getText())
tx = " ".join(text.split('\n'))   
print(tx)

In [ ]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}-{ent.text}')